In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime
import numpy as np

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.preprocessing import LabelEncoder
import numpy as np
import folium

data = pd.read_csv('/content/Madis_Locations.csv')

def parse_date(date_str):
    try:
        return pd.to_datetime(date_str, format='%m/%d/%Y')
    except ValueError:
        return pd.to_datetime(date_str, errors='coerce')

# Convert 'Date' and 'Arrival Time' to datetime
data['Date'] = data['Date'].apply(parse_date)
data['Arrival Time'] = pd.to_datetime(data['Arrival Time'], format='%I:%M %p', errors='coerce')

# Drop rows with missing 'Date', 'Arrival Time', 'lat', or 'lon'
data = data.dropna(subset=['Date', 'Arrival Time', 'lat', 'lon'])

# extracting features from 'Date' and 'Arrival Time'
data['DOW'] = data['Date'].dt.dayofweek
data['Hour'] = data['Arrival Time'].dt.hour

# combining lat and lon to create a location identifier
data['LocationID'] = data['lat'].astype(str) + ',' + data['lon'].astype(str)

# defining features and target
X = data[['DOW', 'Hour']]
y = data['LocationID']
locations = data[['LocationID', 'Location', 'lat', 'lon']].drop_duplicates().set_index('LocationID')


# splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# trainning the model
location_model = RandomForestClassifier()
location_model.fit(X_train, y_train)

# predicting on the test set
y_pred = location_model.predict(X_test)

# evaluating model
accuracy = accuracy_score(y_test, y_pred)
print(f'Location Prediction Accuracy: {accuracy}')

# decoding predicted and actual labels back to LocationID
y_test_decoded = le.inverse_transform(y_test)
y_pred_decoded = le.inverse_transform(y_pred)

# calcing MSE for lat lon
lat_test = np.array([float(loc.split(',')[0]) for loc in y_test_decoded])
lon_test = np.array([float(loc.split(',')[1]) for loc in y_test_decoded])
lat_pred = np.array([float(loc.split(',')[0]) for loc in y_pred_decoded])
lon_pred = np.array([float(loc.split(',')[1]) for loc in y_pred_decoded])

lat_mse = mean_squared_error(lat_test, lat_pred)
lon_mse = mean_squared_error(lon_test, lon_pred)

print(f'Latitude MSE: {lat_mse}')
print(f'Longitude MSE: {lon_mse}')

# Prediction function
def predict_location(dow, hour):
    input_data = pd.DataFrame([[dow, hour]], columns=['DOW', 'Hour'])
    location_id_pred = location_model.predict(input_data)[0]
    location_id_pred_decoded = le.inverse_transform([location_id_pred])[0]
    location = locations.loc[location_id_pred_decoded]
    lat_pred = location['lat']
    lon_pred = location['lon']
    address_pred = location['Location']
    return lat_pred, lon_pred, address_pred

# Function to display the map
def display_map(lat, lon, address):
    # Create a map centered around the predicted location
    m = folium.Map(location=[lat, lon], zoom_start=20)
    # Add a marker for the predicted location
    folium.Marker([lat, lon], popup=address).add_to(m)
    # Save the map as an HTML file and display it
    map_filename = f'predicted_location_at_{address}.html'
    m.save(map_filename)
    return map_filename

# User inputs for day of the week and hour
print(f'Enter the day of the week (integer from 0 to 6):\n0 = Monday\n1 = Tuesday\n2 = Wednesday\n3 = Thursday\n4 = Friday\n5 = Saturday\n6 = Sunday')
dow = int(input())
print("Enter the hour of the day (0-23): ")
hour = int(input())

pred_lat, pred_lon, pred_address = predict_location(dow, hour)
print(f'On day {dow}, at hour {hour}, the predicted location is ({pred_lat}, {pred_lon}), {pred_address}.')

map_filename = display_map(pred_lat, pred_lon, pred_address)
